In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")


In [2]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"
#define GPIO0 0
#define GPIO1 1
#define GPIO2 2

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    int a = gpio_read(pin_in);
    pyprintf("pin %d --- val is %d\n", pin, a);
    return a;
    
    
}

void reset_all_gpios(){

    pyprintf("reset PMOD \n");
    for(int pin=0; pin < 6; pin++) {
        write_gpio(pin, 0);
    }
    
}


In [3]:
#Reset all gpios
reset_all_gpios()

In [4]:
import time

def emulatePwm(frequency, dutycycle, color):
    period = 1 / frequency
    newDutyCycle = dutycycle / 100
    delay = period * newDutyCycle
    
    print("on delay: ", delay)
    print("off delay:", period - delay)
    count = frequency
    
    while count:
        write_gpio(0, 0x1);
        time.sleep(delay);
        read_gpio(0);
        
        
        write_gpio(0, 0x0);
        time.sleep(period - delay);
    
        count -= 1

In [5]:
#Set -ve pin to ground (0)

i = 1

while i < 1000:
    emulatePwm(1, 20, 4)
    i = i+1

on delay:  0.2
off delay: 0.8
reset PMOD 
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay

on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off d

pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
o

on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off d

pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
o

on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off delay: 0.8
pin 0 --- val is 1
on delay:  0.2
off d